In [23]:
!pip install transformers

In [1]:
!pip install tensorflow
!pip install sklearn
!pip install nltk
!pip install sentencepiece

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [2]:
import pandas as pd
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
import nltk
import string
import re

# Download NLTK data for stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

# Import stopwords
from nltk.corpus import stopwords


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [3]:
# Load the dataset
data = pd.read_csv('/content/translated_train.csv')

# Preprocess function
def preprocess_text(text):
    # Check if the text is a string before processing
    if isinstance(text, str):
        # Convert to lowercase
        text = text.lower()
        # Remove punctuation
        text = text.translate(str.maketrans('', '', string.punctuation))
        # Remove numbers
        text = re.sub(r'\d+', '', text)
        # Tokenize and remove stopwords
        stop_words = set(stopwords.words('indonesian'))
        words = word_tokenize(text)
        text = ' '.join([word for word in words if word not in stop_words])
        return text
    else:
        # Handle non-string values, e.g., by returning an empty string or NaN
        return ''  # or pd.NA

# Apply preprocessing to the context and response columns
data['processed_context'] = data['translated_context'].apply(preprocess_text)
data['processed_response'] = data['translated_response'].apply(preprocess_text)

# Show the preprocessed data
data[['translated_context', 'processed_context', 'translated_response', 'processed_response']].head()

,translated_context,processed_context,translated_response,processed_response
0,Saya mengalami beberapa hal dengan perasaan da...,mengalami perasaan tidur memikirkan berharga m...,"Jika semua orang berpikir Anda tidak berharga,...",orang berpikir berharga menemukan orang bergau...
1,Saya mengalami beberapa hal dengan perasaan da...,mengalami perasaan tidur memikirkan berharga m...,"Halo, dan terima kasih atas pertanyaan Anda da...",halo terima kasih mencari nasihat perasaan ber...
2,Saya mengalami beberapa hal dengan perasaan da...,mengalami perasaan tidur memikirkan berharga m...,Hal pertama yang saya sarankan adalah mendapat...,sarankan tidur butuhkan memengaruhi berpikir m...
3,Saya mengalami beberapa hal dengan perasaan da...,mengalami perasaan tidur memikirkan berharga m...,Terapi sangat penting bagi mereka yang merasa ...,terapi depresi berharga mengalami kekhawatiran...
4,Saya mengalami beberapa hal dengan perasaan da...,mengalami perasaan tidur memikirkan berharga m...,Pertama -tama saya ingin memberi tahu Anda bah...,tama perasaan membantu mengubah perasaan mengu...


In [9]:
import numpy as np
import torch
from sklearn.metrics.pairwise import cosine_similarity

# Assuming 'model', 'tokenizer', and 'device' (CPU or GPU) are already defined

# Function to get batch embeddings
def get_batch_embeddings(texts, batch_size=16):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]

        # Tokenization and moving to the correct device (GPU/CPU)
        inputs = tokenizer(batch, return_tensors='pt', padding=True, truncation=True, max_length=128)

        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            # Forward pass through the model
            outputs = model(**inputs)

        last_hidden = outputs.last_hidden_state
        attention_mask = inputs['attention_mask']

        # Apply mean pooling (sum and average over tokens)
        mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden.size()).float()
        summed = torch.sum(last_hidden * mask_expanded, 1)
        counts = torch.clamp(mask_expanded.sum(1), min=1e-9)
        mean_pooled = summed / counts

        embeddings.append(mean_pooled.cpu().numpy())

    return np.vstack(embeddings)

# Function to get a single query embedding
def get_query_embedding(query):
    preprocessed = preprocess_text_indonesian(query)  # Preprocessing function for Indonesian text
    inputs = tokenizer(preprocessed, return_tensors='pt', truncation=True, padding=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        # Forward pass through the model
        outputs = model(**inputs)

    last_hidden = outputs.last_hidden_state
    attention_mask = inputs['attention_mask']

    # Apply mean pooling
    mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden.size()).float()
    summed = torch.sum(last_hidden * mask_expanded, 1)
    counts = torch.clamp(mask_expanded.sum(1), min=1e-9)
    mean_pooled = summed / counts

    # Return as 2D array for compatibility with cosine similarity
    return mean_pooled.cpu().numpy().reshape(1, -1)

# Function to get semantic response from the chatbot
def get_semantic_chatbot_response_with_fallback(user_query, df, context_embeddings, similarity_threshold=0.3):
    # Preprocess query
    preprocessed_query = preprocess_text_indonesian(user_query)
    print(f"Query pengguna setelah pra-pemrosesan: '{preprocessed_query}'")

    if not preprocessed_query.strip():
        return "Maaf, pertanyaan Anda kosong atau tidak dapat dipahami."

    # Get the query embedding
    embedding = get_query_embedding(preprocessed_query)

    # Compute cosine similarity between query embedding and context embeddings
    similarity_scores = cosine_similarity(embedding, context_embeddings)[0]
    max_score = np.max(similarity_scores)
    print(f"Skor kemiripan tertinggi: {max_score:.4f}")

    # If similarity is below threshold, return fallback response
    if max_score < similarity_threshold:
        print(f"Skor kemiripan ({max_score:.4f}) < threshold ({similarity_threshold})")
        return "Maaf, saya belum memahami pertanyaan Anda dengan baik. Bisa dijelaskan dengan cara lain?"

    # Find the best matching response
    best_idx = np.argmax(similarity_scores)
    response_col = 'translated_response'

    try:
        # Get the response text from the best matching context
        response_text = df.iloc[best_idx][response_col]
        print(f"Indeks respons terbaik: {best_idx}")
        print(f"Respons yang ditemukan: {response_text}")

        # Ensure the response is valid (not empty or non-string)
        if not isinstance(response_text, str) or not response_text.strip():
            return "Maaf, saya tidak menemukan jawaban yang sesuai."
        return response_text
    except KeyError:
        return f"Error: Kolom '{response_col}' tidak ditemukan."
    except Exception as e:
        print(f"Kesalahan saat mengambil respons: {e}")
        return "Maaf, terjadi kesalahan teknis."


In [10]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# Prepare input-output pairs for the training
input_texts = data['processed_context'].values
target_texts = data['processed_response'].values

# Define maximum length of the sequences (you can adjust this)
MAX_SEQ_LENGTH = 50

# Tokenize the input and output sequences
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(input_texts + target_texts)

# Convert texts to sequences
input_seq = tokenizer.texts_to_sequences(input_texts)
target_seq = tokenizer.texts_to_sequences(target_texts)

# Pad sequences to a fixed length
input_seq = tf.keras.preprocessing.sequence.pad_sequences(input_seq, maxlen=MAX_SEQ_LENGTH, padding='post')
target_seq = tf.keras.preprocessing.sequence.pad_sequences(target_seq, maxlen=MAX_SEQ_LENGTH, padding='post')

# Split into training and validation sets
input_train, input_val, target_train, target_val = train_test_split(input_seq, target_seq, test_size=0.2)

# Define model parameters
input_dim = len(tokenizer.word_index) + 1
output_dim = len(tokenizer.word_index) + 1
embedding_dim = 256
hidden_units = 256

# Build the encoder-decoder model using LSTM
inputs = Input(shape=(MAX_SEQ_LENGTH,))
x = Embedding(input_dim=input_dim, output_dim=embedding_dim)(inputs)
x = LSTM(hidden_units, return_state=True)(x)
encoder_outputs, state_h, state_c = x

# Decoder part
decoder_inputs = Input(shape=(MAX_SEQ_LENGTH,))
x2 = Embedding(input_dim=input_dim, output_dim=embedding_dim)(decoder_inputs)
x2 = LSTM(hidden_units, return_sequences=True)(x2, initial_state=[state_h, state_c])
decoder_outputs = Dense(output_dim, activation='softmax')(x2)

# Compile the model
model = Model([inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Model training
model.fit([input_train, target_train], target_train, batch_size=32, epochs=10, validation_data=([input_val, target_val], target_val))


Epoch 1/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 7s 60ms/step - accuracy: 0.2032 - loss: 7.5709 - val_accuracy: 0.2190 - val_loss: 5.8106
Epoch 2/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 10s 57ms/step - accuracy: 0.2257 - loss: 5.6073 - val_accuracy: 0.2309 - val_loss: 5.3530
Epoch 3/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 5s 57ms/step - accuracy: 0.2430 - loss: 5.1987 - val_accuracy: 0.2571 - val_loss: 5.0546
Epoch 4/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 5s 56ms/step - accuracy: 0.2726 - loss: 4.8440 - val_accuracy: 0.3131 - val_loss: 4.6989
Epoch 5/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 5s 56ms/step - accuracy: 0.3366 - loss: 4.4786 - val_accuracy: 0.4545 - val_loss: 4.1714
Epoch 6/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 5s 57ms/step - accuracy: 0.4985 - loss: 3.8473 - val_accuracy: 0.5911 - val_loss: 3.4525
Epoch 7/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 5s 55ms/step - accuracy: 0.6268 - loss: 3.1205 - val_accuracy: 0.6916 - val_loss: 2.7751
Epoch 8/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 5s 56ms/step - accuracy: 0.7096 - loss: 2.4832 - val_accuracy: 0.7503 - 